# Clean and preprocess the GDSC dataset

In [ ]:
from __future__ import annotations

import polars as pl
import pandas as pd
import numpy as np

from pathlib import Path

In [ ]:
data_folder = Path("../../data/raw/GDSC")
output_folder = Path("../../data/processed/GDSC")
output_folder.mkdir(exist_ok=True)

In [ ]:
# load drug info

drug_info = pd.read_csv(
    data_folder / "drug_list_2023_06_23_curation_annotated.csv",
)

drug_info.head()

In [ ]:
# load drug screening data

screen_v2 = pd.read_excel(
    data_folder / "GDSC2_fitted_dose_response_24Jul22.xlsx",
)
screen_v1 = pd.read_excel(
    data_folder / "GDSC1_fitted_dose_response_24Jul22.xlsx",
)

In [ ]:
combined_screen = pd.concat([screen_v1, screen_v2])
combined_screen.head()

In [ ]:
# where possible, choose GDSC2
# if there are two for GDSC2, choose the one with the most cell lines

In [ ]:
# add smiles string annotations

drug_annots = (
    drug_info[["Drug Id", "CanonicalSMILES__PubCHEM", "Title__PubCHEM"]]
    .dropna(subset="CanonicalSMILES__PubCHEM")
    .drop_duplicates(subset=["Drug Id", "CanonicalSMILES__PubCHEM"])
)

combined_screen = combined_screen.merge(
    drug_annots,
    left_on="DRUG_ID",
    right_on="Drug Id",
    how="inner",
)

combined_screen["ID"] = (
    combined_screen[["DRUG_ID", "DATASET"]].astype(str).agg("-".join, axis=1)
)

In [ ]:
cell_line_counts = (
    combined_screen.groupby(
        ["CanonicalSMILES__PubCHEM", "DRUG_ID", "DATASET"]
    )["SANGER_MODEL_ID"]
    .size()
    .to_frame(name="count")
    .reset_index()
    .sort_values(
        ["CanonicalSMILES__PubCHEM", "DATASET", "count"],
        ascending=[True, False, False],
    )
)

cell_line_counts["ID"] = (
    cell_line_counts[["DRUG_ID", "DATASET"]].astype(str).agg("-".join, axis=1)
)

keep_ids = cell_line_counts.drop_duplicates(
    subset=["CanonicalSMILES__PubCHEM"], keep="first"
)["ID"].to_list()

combined_screen = combined_screen[combined_screen["ID"].isin(keep_ids)]

In [ ]:
col_map = {
    "SANGER_MODEL_ID": "model_id",
    "DRUG_ID": "drug_id",
    "DRUG_NAME": "drug_name",
    "Title__PubCHEM": "drug_name",
    "CanonicalSMILES__PubCHEM": "canonical_smiles",
    "LN_IC50": "ln_ic50",
    "AUC": "auc",
    "DATASET": "dataset",
}

combined_screen_clean = (
    combined_screen[list(col_map)].rename(columns=col_map).reset_index()
)

combined_screen_clean.head()

In [ ]:
combined_screen_clean.to_csv(
    output_folder / "ScreenFittedDoseResponse.csv", index=False
)

In [ ]:
drug_info["ID"] = (
    drug_info[["Drug Id", "Datasets"]].astype(str).agg("-".join, axis=1)
)

col_map = {
    "Drug Id": "drug_id",
    "Name": "drug_name",
    "Datasets": "dataset",
    "CanonicalSMILES__PubCHEM": "canonical_smiles",
    "Targets": "targets",
    "Target pathway": "target_pathway",
    "Screening site": "screening_site",
    "PubCHEM__curation": "pubchem_id",
    "Title__PubCHEM": "pubchem_drug_name",
    "MolecularFormula__PubCHEM": "pubchem_molecular_formula",
    "MolecularWeight__PubCHEM": "pubchem_molecular_weight",
    "InChIKey__PubCHEM": "pubchem_inchi_key",
}

drug_info_clean = (
    drug_info[list(col_map)][drug_info["ID"].isin(keep_ids)]
    .rename(columns=col_map)
    .reset_index()
)

drug_info_clean.head()

In [ ]:
drug_info_clean.to_csv(output_folder / "DrugAnnotations.csv", index=False)

# End main
## Additional analyses
## TODO - migrate below to analysis/datasets/gdsc_data_exploration.ipynb

In [ ]:
gdsc_drug_annots = (
    gdsc_drug_list.select(
        ["Drug Id", "CanonicalSMILES__PubCHEM", "Screening site", "Datasets"]
    )
    .unique(subset=["Drug Id", "CanonicalSMILES__PubCHEM"])
    .drop_nulls()
)

gdsc_drug_annots.head()

In [ ]:
# for each drug with a smiles string, I want to choose the drug id with the most associated cell lines

In [ ]:
gdsc_combined_screen_smiles = gdsc_combined_screen.join(
    gdsc_drug_annots,
    left_on="DRUG_ID",
    right_on="Drug Id",
    how="left",
    validate="m:1",
)

gdsc_combined_screen_smiles

## Business rules

For each cell line with duplicate smiles strings, we first try to choose the response from GDSC2 and then we choose the response from SANGER over MGH

In [ ]:
# remove duplicate drug response observations
gdsc_combined_screen_smiles = (
    gdsc_combined_screen_smiles.drop_nulls(subset="CanonicalSMILES__PubCHEM")
    .sort(
        [
            "SANGER_MODEL_ID",
            "CanonicalSMILES__PubCHEM",
            "DATASET",
            "Screening site",
        ],
        descending=[False, False, True, True],
    )
    .unique(
        subset=["SANGER_MODEL_ID", "CanonicalSMILES__PubCHEM"],
        keep="first",
        maintain_order=True,
    )
)

gdsc_combined_screen_smiles.head()

In [ ]:
gdsc_combined_screen_smiles.select(
    [
        pl.col("SANGER_MODEL_ID").alias("model_id"),
        pl.col("DRUG_ID").alias("drug_id"),
        pl.col("CanonicalSMILES__PubCHEM").alias("canonical_smiles"),
        pl.col("LN_IC50").alias("ln_ic50"),
        pl.col("AUC").alias("auc"),
        pl.col("DATASET").alias("dataset"),
        pl.col("Screening site").alias("screening_site"),
    ]
).write_csv(gdsc_output_folder / "ScreenFittedDoseResponse.csv")

In [ ]:
left_cols = [
    "DRUG_ID",
    "DATASET",
    "CanonicalSMILES__PubCHEM",
    "Screening site",
]
right_cols = [
    "Drug Id",
    "Datasets",
    "CanonicalSMILES__PubCHEM",
    "Screening site",
]
gdsc_combined_screen_smiles.select(left_cols).unique().join(
    gdsc_drug_list,
    left_on=left_cols,
    right_on=right_cols,
    how="inner",
)["CanonicalSMILES__PubCHEM"].n_unique()

In [ ]:
# assess correlation between GDSCv1 and GDSCv2 responses

gdsc_v1_ic50 = (
    gdsc_v1_screen[["DRUG_ID", "SANGER_MODEL_ID", "LN_IC50"]]
    .join(
        drug_to_smiles,
        left_on="DRUG_ID",
        right_on="Drug Id",
        how="inner",
        validate="m:1",
    )
    .rename({"LN_IC50": "LN_IC50_v1", "DRUG_ID": "DRUG_ID_v1"})
)

gdsc_v2_ic50 = (
    gdsc_v2_screen[["DRUG_ID", "SANGER_MODEL_ID", "LN_IC50"]]
    .join(
        drug_to_smiles,
        left_on="DRUG_ID",
        right_on="Drug Id",
        how="inner",
        validate="m:1",
    )
    .rename({"LN_IC50": "LN_IC50_v2", "DRUG_ID": "DRUG_ID_v2"})
)

In [ ]:
from scipy import stats

def pearsonr(s: pl.Struct) -> float:
    if s[0].shape[0] > 3:
        result = stats.pearsonr(s[0], s[1])
        return result[0]
    return np.nan

In [ ]:
gdsc_v1_v2_screen_common = gdsc_v2_ic50.join(
    gdsc_v1_ic50,
    on=["CanonicalSMILES__PubCHEM", "SANGER_MODEL_ID"],
    how="inner",
)

gdsc_v1_v2_screen_common.head()

In [ ]:
gdsc_v1_v2_corr = (
    gdsc_v1_v2_screen_common.groupby("CanonicalSMILES__PubCHEM")
    .agg(pl.apply(["LN_IC50_v2", "LN_IC50_v1"], pearsonr).alias("pearson"))
    .sort("pearson", descending=True)
)

gdsc_v1_v2_corr.head()

In [ ]:
gdsc_v1_v2_corr["pearson"].describe()